# Load data

In [42]:

#from nltk.corpus import twitter_samples

#positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
#negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

In [53]:
#target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
from nltk.tokenize import word_tokenize
import pandas as pd

df = pd.read_csv('training_data.csv', delimiter=',')

positive_tweet_tokens = []
negative_tweet_tokens = []

positive_row_count=0
negative_row_count=0
for row in df.values:
    if row[0]==0 and negative_row_count<200000:
        negative_row_count+=1
        negative_tweet_tokens.append(word_tokenize(row[5]))
    elif row[0]==4 and positive_row_count<200000:
        positive_row_count+=1
        positive_tweet_tokens.append(word_tokenize(row[5]))

print(len(positive_tweet_tokens))
print(len(negative_tweet_tokens))

200000
200000


# Removing Noise from the Data

In [54]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words and token not in ["...","http"]:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

# Clean the Positive and Negative Data

In [55]:
from nltk.corpus import stopwords

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

stop_words = stopwords.words('english')

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    

print(positive_tweet_tokens[0])
print(positive_cleaned_tokens_list[0])

['I', 'LOVE', '@', 'Health4UandPets', 'u', 'guys', 'r', 'the', 'best', '!', '!']
['love', 'health4uandpets', 'u', 'guy', 'r', 'best']


# Determining Word Density

In [56]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)

# Testing Word Density

In [57]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[("'s", 24354), ('good', 19020), ('get', 18009), ('day', 15611), ('go', 15286), ("'m", 13967), ("n't", 13567), ('love', 12924), ('quot', 11691), ('..', 11671)]


# Converting Tokens to a Dictionary

In [58]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)


# Splitting the Dataset for Training and Testing the Model

In [59]:
import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]


negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:320000]
test_data = dataset[320000:]
print(len(train_data))
print(len(test_data))

320000
80000


In [62]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.7552125
Most Informative Features
               depressed = True           Negati : Positi =     37.2 : 1.0
               toothache = True           Negati : Positi =     35.6 : 1.0
                     bom = True           Positi : Negati =     29.7 : 1.0
                coughing = True           Negati : Positi =     27.6 : 1.0
                    roni = True           Negati : Positi =     27.6 : 1.0
          disappointment = True           Negati : Positi =     27.0 : 1.0
                 unhappy = True           Negati : Positi =     27.0 : 1.0
                nauseous = True           Negati : Positi =     24.3 : 1.0
                   ariel = True           Positi : Negati =     23.0 : 1.0
                  booooo = True           Negati : Positi =     22.6 : 1.0
None


In [68]:
from nltk.tokenize import word_tokenize

custom_tweet = "I am joyful"
custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive
